<a href="https://colab.research.google.com/github/warkechetan/BNet/blob/master/reliance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#INITIALIZE VARIABLES
smaLenght = 50  #length of sma window
df=pd.read_csv('RELIANCE.NS.csv')
adjustedClose=df['adjusted_close']
volume=df['volume']
volumeR=volume.iloc[::-1].values

adjustedClose=adjustedClose.iloc[::-1].values
adjustedClose=pd.DataFrame(adjustedClose)
volumeR=pd.DataFrame(volumeR)
print(len(volumeR))

meanAdjustedClose=adjustedClose.rolling(window=smaLenght).mean()

print(adjustedClose[0][5])
#print(meanAdjustedClose)

#print(volumeR[0])


4855
19.9262


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
def roi(bo,stockPrice,amt,stocks):
  
  if bo==1:
    stocks=int(amt/stockPrice)
    amt=amt-stocks*stockPrice
    #print(" amout buy",amt)
    
  else:
    amt=amt+stocks*stockPrice
    
    #print(" amout sell",amt)
    stocks=0
  return amt,stocks



In [0]:
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense,Dropout

def buysell(meanAdjustedCloseF,df,k,percentage):
  flag=0
  amt=10000
  stocks=0
  
  res=[0*x for x in range(smaLenght,len(meanAdjustedClose))]
  for i in range(smaLenght,len(res)):
    
    if meanAdjustedCloseF[i]<df[i] and flag==0:
        res[i]=1
        flag=1
        amt,stocks=roi(1,df[i],amt,stocks)
        
        
    elif meanAdjustedCloseF[i]>df[i] and flag==0:
        res[i]=0
    elif meanAdjustedCloseF[i]>df[i] and flag==1:
        res[i]=-1
        flag=0
        amt,stocks=roi(0,df[i],amt,stocks)
  res=pd.DataFrame(res)
  #print(res[0])
  target=[0*x for x in range(0,len(res))]
  for i in range(0,len(target)):
    if res[0][i]==1 or res[0][i]==-1 :
    #print("working1",adjustedClose[i])
      temp=((df[i+k]-df[i])/df[i])*100
      #print("working")
      if temp>percentage:
        target[i]=1
      elif temp<-percentage :
        target[i]=-1
      else:
        target[i]=0

  target=pd.DataFrame(target)

  
  return res,target

#print(target[0])
k=2             #No of days
percentage=1    #percentage for buy/sell                 
res,target=buysell(meanAdjustedClose[0],adjustedClose[0],k,percentage)
#target1=pd.DataFrame(target1)



In [0]:
import pandas as pd
a=pd.DataFrame([1,2,3])
a.item()


AttributeError: ignored

In [23]:

import tensorflow as tf
import numpy as np
finaldf=pd.read_csv('nn.csv')
scaler=StandardScaler()
X=finaldf[['price','volume','result','mean']]
from sklearn import preprocessing
y=finaldf[['target']]
adjustedCloseN=X['price']
adjustedCloseN=np.array(adjustedCloseN.values)
scaler = preprocessing.MinMaxScaler()
adjustedCloseN = scaler.fit_transform(adjustedCloseN[:,np.newaxis])
volume = X['volume']
volume = np.array(volume.values)
scaler = preprocessing.MinMaxScaler()
volume = scaler.fit_transform(volume[:,np.newaxis])
adjustedCloseN=pd.DataFrame(adjustedCloseN)
volume=pd.DataFrame(volume)
result=X['result']
mean=X['mean']
mean=np.array(mean.values)
mean=scaler.fit_transform(mean[:,np.newaxis])
print(len(mean))
#print(result)
#result.replace(to_replace="-1",value="001")
#result.replace(to_replace="0",value="010")
#result.replace(to_replace="1",value="100")
#print(result)
adjustedCloseN[1]=volume[0]
adjustedCloseN[2]=mean
adjustedCloseN[3]=X['result']
X=adjustedCloseN
print(X)

4803
             0         1         2  3
0     0.000000  0.223842  0.000818  0
1     0.000817  0.126496  0.000786  0
2     0.000729  0.233986  0.000752  0
3     0.000871  0.151454  0.000705  0
4     0.001127  0.109229  0.000649  0
5     0.001013  0.094932  0.000579  0
6     0.000997  0.058425  0.000513  0
7     0.001893  0.091733  0.000466  0
8     0.002862  0.155634  0.000425  0
9     0.002426  0.227641  0.000394  0
10    0.003437  0.380256  0.000350  0
11    0.004529  0.436423  0.000334  0
12    0.005095  0.472942  0.000337  0
13    0.003872  0.299683  0.000345  0
14    0.003868  0.557846  0.000335  0
15    0.006387  0.359675  0.000311  0
16    0.007715  0.258833  0.000341  0
17    0.008341  0.489724  0.000405  0
18    0.007791  0.216162  0.000467  0
19    0.006987  0.382373  0.000514  0
20    0.005613  0.004988  0.000555  0
21    0.004975  0.352866  0.000560  0
22    0.005059  0.194372  0.000547  0
23    0.005812  0.360885  0.000533  0
24    0.006977  0.429419  0.000533  0
25    0

In [24]:
import keras
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1)
yt=keras.utils.to_categorical(y_train,3)
print(yt)
model = Sequential()
model.add(Dense(4, input_dim=4, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(3,activation='softmax'))
opt=keras.optimizers.Adam(lr=1e-3,decay=1e-5)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, yt, epochs=15, batch_size=40)
#accuracy = model.evaluate(X_test,y_test)
#accuracy=accuracy_score(res1,target1)
#print(accuracy)


W0819 17:21:37.984888 140453661820800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0819 17:21:38.031103 140453661820800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0819 17:21:38.037980 140453661820800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0819 17:21:38.055041 140453661820800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0819 17:21:38.064667 

[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


W0819 17:21:38.185794 140453661820800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0819 17:21:38.192790 140453661820800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0819 17:21:38.199002 140453661820800 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/15
4322/4322 [==============================] - 5s 1ms/step - loss: 0.2392 - acc: 0.9125
Epoch 2/15
4322/4322 [==============================] - 1s 118us/step - loss: 0.0615 - acc: 0.9754
Epoch 3/15
4322/4322 [==============================] - 1s 120us/step - loss: 0.0533 - acc: 0.9768
Epoch 4/15
4322/4322 [==============================] - 1s 125us/step - loss: 0.0533 - acc: 0.9764
Epoch 5/15
4322/4322 [==============================] - 0s 107us/step - loss: 0.0531 - acc: 0.9762
Epoch 6/15
4322/4322 [==============================] - 0s 101us/step - loss: 0.0474 - acc: 0.9773
Epoch 7/15
4322/4322 [==============================] - 0s 104us/step - loss: 0.0479 - acc: 0.9776
Epoch 8/15
4322/4322 [==============================] - 0s 108us/step - loss: 0.0478 - acc: 0.9772
Epoch 9/15
4322/4322 [==============================] - 0s 105us/step - loss: 0.0471 - acc: 0.9775
Epoch 10/15
4322/4322 [==============================] - 0s 102us/step - loss: 0.0505 - acc: 0.9769
Epoch 11/15

In [0]:
predicted = model.predict(X_train)
predicted

array([[0.31482074, 0.39035788, 0.29482144],
       [0.3624588 , 0.3095768 , 0.3279644 ],
       [0.31482074, 0.39035788, 0.29482144],
       ...,
       [0.32280004, 0.3787336 , 0.29846635],
       [0.31482074, 0.39035788, 0.29482144],
       [0.31482074, 0.39035788, 0.29482144]], dtype=float32)

In [0]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
"""
actual =  
predicted = [1, 0, 0, 1, 0, 0, 1, 1, 1, 0] 
results = confusion_matrix(actual, predicted) 
print 'Confusion Matrix :'
print(results) 
print 'Accuracy Score :',accuracy_score(actual, predicted) 
print 'Report : '
print classification_report(actual, predicted) 
"""


"\nactual =  \npredicted = [1, 0, 0, 1, 0, 0, 1, 1, 1, 0] \nresults = confusion_matrix(actual, predicted) \nprint 'Confusion Matrix :'\nprint(results) \nprint 'Accuracy Score :',accuracy_score(actual, predicted) \nprint 'Report : '\nprint classification_report(actual, predicted) \n"